In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [3]:
import tensorflow as tf

2024-06-08 23:19:10.243789: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-08 23:19:10.858891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yw/miniconda3/envs/tf/lib/
2024-06-08 23:19:10.859016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yw/miniconda3/envs/tf/lib/
2024-06-08 23:19:10.859025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries

In [4]:
import tensorflow.keras.backend as K

In [5]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    print(e)

# data preprocessing

In [7]:
df = pd.read_csv('edge_images/preprocessed.csv')

## preprocessing

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53416 entries, 0 to 53415
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            53416 non-null  object 
 1   Sex           53416 non-null  object 
 2   Height        53416 non-null  object 
 3   Weight        53416 non-null  object 
 4   Hair          53416 non-null  object 
 5   Eyes          53416 non-null  object 
 6   Race          53416 non-null  object 
 7   Sex Offender  53416 non-null  bool   
 8   Offense       53416 non-null  object 
 9   Height_cm     53416 non-null  float64
 10  Weight_kg     53416 non-null  float64
dtypes: bool(1), float64(2), object(8)
memory usage: 4.1+ MB


In [9]:
df.drop(columns=['Height', 'Weight', 'Sex Offender', 'Offense', 'Hair', 'Eyes'], inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53416 entries, 0 to 53415
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         53416 non-null  object 
 1   Sex        53416 non-null  object 
 2   Race       53416 non-null  object 
 3   Height_cm  53416 non-null  float64
 4   Weight_kg  53416 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.0+ MB


In [11]:
data = pd.get_dummies(df, columns=['Sex', 'Race'])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53416 entries, 0 to 53415
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  53416 non-null  object 
 1   Height_cm           53416 non-null  float64
 2   Weight_kg           53416 non-null  float64
 3   Sex_Female          53416 non-null  bool   
 4   Sex_Male            53416 non-null  bool   
 5   Race_Amer Indian    53416 non-null  bool   
 6   Race_Asian          53416 non-null  bool   
 7   Race_Bi-Racial      53416 non-null  bool   
 8   Race_Black          53416 non-null  bool   
 9   Race_Hispanic       53416 non-null  bool   
 10  Race_Not Available  53416 non-null  bool   
 11  Race_White          53416 non-null  bool   
dtypes: bool(9), float64(2), object(1)
memory usage: 1.7+ MB


In [13]:
train_data = data[:int(0.8 * len(data))]  # 80% of data for training
val_data = data[int(0.8 * len(data)):int(0.9 * len(data))]  # next 10% of data for validation
test_data = data[int(0.9 * len(data)):]  # last 10% of data for testing

train_data = train_data[~train_data['Race_Not Available']]
val_data = val_data[~val_data['Race_Not Available']]
test_data = test_data[~test_data['Race_Not Available']]

# convert to tuple
train_data = ('edge_images/front/front/'+train_data['ID'] + '.jpg',
              'edge_images/side/side/'+train_data['ID'] + '.jpg',
               tf.constant(train_data.drop(columns=['ID', 'Weight_kg']).values, dtype=tf.float32),
               tf.constant(train_data['Weight_kg'].values, dtype=tf.float32))
val_data = ('edge_images/front/front/'+val_data['ID'] + '.jpg',
            'edge_images/side/side/'+val_data['ID'] + '.jpg',
             tf.constant(val_data.drop(columns=['ID', 'Weight_kg']).values, dtype=tf.float32),
             tf.constant(val_data['Weight_kg'].values, dtype=tf.float32))
test_data = ('edge_images/front/front/'+test_data['ID'] + '.jpg', 
             'edge_images/side/side/'+test_data['ID'] + '.jpg', 
             tf.constant(test_data.drop(columns=['ID', 'Weight_kg']).values, dtype=tf.float32),
             tf.constant(test_data['Weight_kg'].values, dtype=tf.float32))

In [30]:
BATCH_SIZE = 32
N_TRAIN = len(train_data[0])
N_TRAIN

42708

In [31]:
# read images file by file path
def read_imgs(file_paths):
    imgs = tf.TensorArray(tf.float32, size=BATCH_SIZE)
    i=0
    for file_path in file_paths:
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (224, 224))
        img = preprocess_input(img)
        imgs = imgs.write(i, img)
        i+=1
    return imgs.stack()

In [32]:
# run eagerly
# tf.config.run_functions_eagerly(True)
mapper = lambda front, side, x, y: ((read_imgs(front), read_imgs(side), x), y)
train_set = tf.data.Dataset.from_tensor_slices(train_data).shuffle(N_TRAIN).batch(BATCH_SIZE, drop_remainder=True).map(mapper).prefetch(1)
val_set = tf.data.Dataset.from_tensor_slices(val_data).batch(BATCH_SIZE, drop_remainder=True).map(mapper).prefetch(1)
test_set = tf.data.Dataset.from_tensor_slices(test_data).batch(BATCH_SIZE, drop_remainder=True).map(mapper).prefetch(1)

In [33]:
# create model
base_model_front = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
base_model_front._name = 'EfficientNetB0_front'


model_front = tf.keras.Sequential([
    base_model_front,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])
model_front.load_weights('models/edge_model/front_edge.h5')

for layer in base_model_front.layers:
    layer._name = layer.name + "_front"

for layer in model_front.layers:
    layer._name = layer.name + "_front"

model_front.input._name = model_front.input.name + "_front"

In [34]:
# create model
base_model_side = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
base_model_side._name = 'EfficientNetB0_side'


model_side = tf.keras.Sequential([
    base_model_side,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])
model_side.load_weights('models/edge_model/side_edge_5')

for layer in base_model_side.layers:
    layer._name = layer.name + "_side"

for layer in model_side.layers:
    layer._name = layer.name + "_side"

model_side.input._name = model_side.input.name + "_side"

In [35]:
model_structured = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(train_data[2].shape[1:])),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])
model_structured.load_weights('models/structured_model')

for layer in model_structured.layers:
    layer._name = layer.name + "_structured"

In [36]:
_x = tf.keras.layers.concatenate([model_front.layers[-2].output, model_side.layers[-2].output, model_structured.layers[-3].output])
top_model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])
_x = top_model(_x)

In [37]:
model = tf.keras.Model(inputs=[model_front.input, model_side.input, model_structured.input], outputs=_x)

# first train

In [22]:
base_model_front.trainable = False
base_model_side.trainable = False
model_structured.trainable = False

model.compile(optimizer="adam", loss='mse', metrics=['mae'])

In [52]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 batch_normalization_20_structu  (None, 10)          40          ['input_10[0][0]']               
 red (BatchNormalization)                                                                         
                                                                                                  
 dense_27_structured (Dense)    (None, 64)           704         ['batch_normalization_20_structur
                                                                 ed[0][0]']                       
                                                                                            

In [27]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [54]:
model.fit(train_set, validation_data=val_set, epochs=200, callbacks = [early_stopping])

Epoch 1/200
333/333 [==============================] - 121s 332ms/step - loss: 7293.8018 - mae: 84.4700 - val_loss: 6242.6055 - val_mae: 77.9789
Epoch 2/200
333/333 [==============================] - 112s 335ms/step - loss: 5884.3955 - mae: 75.8595 - val_loss: 4585.9751 - val_mae: 66.5554
Epoch 3/200
333/333 [==============================] - 111s 332ms/step - loss: 3802.4021 - mae: 60.5107 - val_loss: 2794.1597 - val_mae: 51.4487
Epoch 4/200
333/333 [==============================] - 112s 334ms/step - loss: 1891.9243 - mae: 41.8098 - val_loss: 1190.3439 - val_mae: 32.3278
Epoch 5/200
333/333 [==============================] - 112s 334ms/step - loss: 714.1335 - mae: 24.2031 - val_loss: 418.1253 - val_mae: 17.1249
Epoch 6/200
333/333 [==============================] - 111s 331ms/step - loss: 243.1752 - mae: 12.5532 - val_loss: 182.7221 - val_mae: 10.1028
Epoch 7/200
333/333 [==============================] - 109s 325ms/step - loss: 129.6695 - mae: 8.6903 - val_loss: 145.1949 - val_mae: 

In [23]:
import requests
def send_telegram_message(bot_token, chat_id, message):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": message}
    response = requests.post(url, data=payload)
    return response.json()

bot_token = '7217981545:AAE4RgPEDfmRVxGOBtegWEfWIAo4V06ZgVU'
chat_id = '6183060417'
message = 'Your Training is finished!'
# print(send_telegram_message(bot_token, chat_id, message))


In [ ]:
model.evaluate(test_set)

In [59]:
model.save_weights('models/edge_model/ensemble_0')

In [25]:
import tensorflow as tf

def reset_keras():
    tf.keras.backend.clear_session()
    print("Keras session cleared.")


In [38]:
steps_of_layers = 20
training_layers = 160
best_result = [100000, 100000]
save_index = 3

while (training_layers < len(base_model_front.layers)):
    base_model_front.trainable = True
    base_model_side.trainable = True
    model_structured.trainable = True

    if (training_layers + steps_of_layers < len(base_model_front.layers)):
        for layer in base_model_front.layers:
            layer.trainable = False
        for layer in base_model_side.layers:
            layer.trainable = False
        for layer in base_model_front.layers[-training_layers:]:
            layer.trainable = True
        for layer in base_model_side.layers[-training_layers:]:
            layer.trainable = True

    print("model layers updated.")
    
    model.load_weights('models/edge_model/ensemble_'+str(save_index))

    print("model weight loaded")

    model.compile(optimizer="adam", loss='mse', metrics=['mae'])
    K.set_value(model.optimizer.learning_rate, 0.1*model.optimizer.learning_rate)

    print("model compiled")
    
    model.fit(train_set, validation_data=val_set, epochs=100, callbacks = [early_stopping])

    result = model.evaluate(test_set)
    print(result)

    message = 'Your Training Layers : ' +str(training_layers) + ' Val_loss, mae: ' + str(result[0])+' '+str(result[1])
    print(send_telegram_message(bot_token, chat_id, message))
    
    if (best_result[0] > result[0]) :  #val_loss가 감소했다면
        print("previous best : ", best_result, ' new best : ', result)
        save_index += 1
        model.save_weights('models/edge_model/ensemble_'+str(save_index))
        print('model saved.')
        best_result = result
        
    reset_keras()

    training_layers = training_layers + steps_of_layers

model layers updated.
model weight loaded
model compiled
Epoch 1/100
1334/1334 [==============================] - 241s 170ms/step - loss: 96.4962 - mae: 7.6185 - val_loss: 145.6432 - val_mae: 8.9011
Epoch 2/100
1334/1334 [==============================] - 224s 168ms/step - loss: 93.3167 - mae: 7.5026 - val_loss: 147.4526 - val_mae: 9.0000
Epoch 3/100
1334/1334 [==============================] - 225s 168ms/step - loss: 90.1783 - mae: 7.3856 - val_loss: 147.6961 - val_mae: 8.9621
Epoch 4/100
1334/1334 [==============================] - 224s 168ms/step - loss: 87.4850 - mae: 7.2949 - val_loss: 149.2558 - val_mae: 9.0472
Epoch 5/100
1334/1334 [==============================] - 225s 169ms/step - loss: 85.3003 - mae: 7.1908 - val_loss: 151.3165 - val_mae: 9.1623
Epoch 6/100
166/166 [==============================] - 13s 77ms/step - loss: 145.9177 - mae: 8.9109
[145.917724609375, 8.910933494567871]
{'ok': True, 'result': {'message_id': 13, 'from': {'id': 7217981545, 'is_bot': True, 'first_nam

2024-06-09 01:34:56.848466: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 150.0KiB (rounded to 153600)requested by op model/EfficientNetB0_front_front/block4b_expand_conv_front/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-06-09 01:34:56.858873: W tensorflow/core/common_runtime/bfc_allocator.cc:491] ******************xx********************************************************************************
2024-06-09 01:34:56.858962: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at conv_ops.cc:1077 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[480,80,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
2024-06-09 01:34:56.859309: W tensorflow/core/common_runtime/bfc_allocator.cc:479] All

ResourceExhaustedError: Graph execution error:

Detected at node 'model/EfficientNetB0_side_side/block4a_expand_conv_side/Conv2D' defined at (most recent call last):
    File "/home/yw/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/yw/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/yw/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/yw/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/yw/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/yw/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_for

In [27]:
base_model_front.trainable = True
base_model_side.trainable = True
model_structured.trainable = True

for layer in base_model_front.layers:
    layer.trainable = False
for layer in base_model_side.layers:
    layer.trainable = False
for layer in base_model_front.layers[-20:]:
    layer.trainable = True
for layer in base_model_side.layers[-20:]:
    layer.trainable = True

model.compile(optimizer="adam", loss='mse', metrics=['mae'])
K.set_value(model.optimizer.learning_rate, 0.1*model.optimizer.learning_rate)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 batch_normalization_structured  (None, 10)          40          ['input_3[0][0]']                
  (BatchNormalization)                                                                            
                                                                                                  
 dense_2_structured (Dense)     (None, 64)           704         ['batch_normalization_structured[
                                                                 0][0]']                          
                                                                                              

In [28]:
model.fit(train_set, validation_data=val_set, epochs=3)

Epoch 1/3
435/435 [==============================] - 141s 305ms/step - loss: 93.4836 - mae: 7.3949 - val_loss: 78.6593 - val_mae: 6.6804
Epoch 2/3
435/435 [==============================] - 131s 300ms/step - loss: 79.7029 - mae: 6.8621 - val_loss: 73.8960 - val_mae: 6.5231
Epoch 3/3
435/435 [==============================] - 131s 301ms/step - loss: 73.2586 - mae: 6.6244 - val_loss: 68.1074 - val_mae: 6.2113


In [41]:
model.load_weights('models/edge_model/ensemble_2')

In [42]:
model.evaluate(test_set)

166/166 [==============================] - 14s 85ms/step - loss: 141.0858 - mae: 8.8559


[141.08580017089844, 8.855900764465332]

In [30]:
model.save_weights('models/ensemble-v1-2')